# LAB 5 MOS: Optimización Multiobjetivo

- Samuel Augusto Hernandez 202213772
- Tomas Alberto Rodriguez 2022128686

## Problema 1: Optimización Multiobjetivo en Distribución de Recursos para Misión Humanitaria

### Formulación del modelo Multiobjetivo

#### Conjuntos

$$ R = \{R_1,R_2,R_3,R_4,R_5\}$$
$$ A = \{A_1,A_2,A_3,A_4\}$$
$$ Z = \{Z_1,Z_2,Z_3,Z_4\}$$

#### Parámetros

Recursos:

Valor de impacto (miles USD/TON):
$$ Va_r: \forall r \in R $$

Peso(TON/unidad):
$$ W_r: \forall r \in R$$

Volumen(m^3/unidad):
$$ V_r: \forall r \in R$$

Disponibilidad(unidades):
$$ D_r: \forall r \in R$$

Aviones:

Capacidad Peso (TON):
$$ CP_a: \forall a \in A$$

Capacidad Volumen (m^3):
$$ CV_a: \forall a \in A$$

Costo Fijo (miles UDS):
$$ CF_a: \forall a \in A$$

Costo Variable (miles UDS/km):
$$ C_a: \forall a \in A$$

Zonas:



#### Variables de decision

#### Funciones objetivo

#### Restricciones

Ejemplo de lab anterior
#### Variable de desicion 
$$ x_{ij} \in \{0,1\} $$
##### Funcion Objetivo
$$ MAX(\sum_{i\in R} \sum_{j\in A} x_{ij} \cdot valor_i) $$
##### Restricciones
$$ \sum_{i\in R} x_{ij} \cdot Peso_i \leq Peso_j \ \forall j \in A$$
$$ \sum_{i\in R} x_{ij} \cdot Volumen_i \leq Volumen_j \ \forall j \in A$$
$$ \sum_{j \in A} x_{ij} \leq 1 \ \forall i \in R$$

- Seguridad Medicamentos

$$ x_{21} = 0$$

- Compatibilidad equipos medicos y agua

$$ j_1 / x_{3j_1} = 1 \wedge j_2 /  x_{4j_2} = 1 \rightarrow j_1 \neq j_2$$

Fin ejemplo lab anterior

#### Modelo de optimización


### Implementación del modelo

In [7]:
#codigo

#### Visualizacion del frente de Pareto

In [8]:
#codigo

### Analisis

## Problema 2: Optimización Multiobjetivo en Planificación de Rutas de Inspección

### Formulación del modelo Multiobjetivo

#### Conjuntos

#### Parámetros

#### Variables de decision

#### Funciones objetivo

#### Restricciones

### Modelo de optimización


### Implementación del modelo

In [9]:
#codigo

#### Visulizacion del frente de Pareto

In [10]:
#codigo

### Analisis